## Artificial Intelligence Assignment 3
## No3. CNN2 Adam Optimizer
## 20132651 Sungjae Lee

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=False,
                                          num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=4,
                                         shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [3]:
## Adam optimizer 를 강의자료대로 학습률 1e-03, 모멘텀 계수 0.9, 가중 이동 평균 계수 0.999를 사용하였습니다.
## 하지만 이 값들 모두 Adam optimizer 의 기본 값과 동일하므로, 작성하지 않아도 괜찮습니다.

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-03, betas = (0.9, 0.999))

### (3) 화면 출력 확인 및 학습이 되고 있는지 서술

In [4]:
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss : %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

## 2번 문제에서의 loss 가 1.226 이었던데 반해 최종 loss 가 1.197 이라는 것은
## Optimizer의 변경만을 통해 loss 를 크게 줄이기는 힘들다는 점을 보여줍니다.
## 하지만 주목해서 볼 부분은 초반의 loss 변화가 2.013 -> 1.730 -> 1.649 로 급감했다는 점입니다.
## 이전까지는 2.299 -> 2.130 -> 1.919 와 같이 매우 더디게 초반 학습이 진행되었던 점을 보면
## loss 의 감소율이 매우 높은 것으로 보여지며, 이는 확실하게 Adam Optimizer 를 사용했기 때문입니다.

## 학습의 진행은 loss가 2.013 부터 1.197 까지 꾸준히 감소하는 것으로 보아 적절하다고 볼 수 있습니다.

[1,  1000] loss : 2.013
[1,  2000] loss : 1.730
[1,  3000] loss : 1.649
[1,  4000] loss : 1.594
[1,  5000] loss : 1.511
[1,  6000] loss : 1.470
[1,  7000] loss : 1.420
[1,  8000] loss : 1.418
[1,  9000] loss : 1.445
[1, 10000] loss : 1.401
[1, 11000] loss : 1.377
[1, 12000] loss : 1.348
[2,  1000] loss : 1.311
[2,  2000] loss : 1.298
[2,  3000] loss : 1.284
[2,  4000] loss : 1.284
[2,  5000] loss : 1.253
[2,  6000] loss : 1.244
[2,  7000] loss : 1.236
[2,  8000] loss : 1.231
[2,  9000] loss : 1.267
[2, 10000] loss : 1.215
[2, 11000] loss : 1.205
[2, 12000] loss : 1.197
Finished Training


### (6) 화면 출력 확인 및 일반화 성능 서술

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images :%d %%'%(100 * correct / total))

## 2번 문제의 신경망과 동일한 55%의 정확도가 나타났습니다.
## 이 점으로 미루어 보아 Optimizer 만을 변경했을 때 성능의 차이가 크게 나타나지는 않는 것 같습니다.
## 그 이유를 간단하게 생각해 보면 현재 loss 가 아주 작아져 과적합된 상황이 아니기 때문에
## Optimizer 를 통해 일반화 성능을 향상시키는 의미가 크게 없어서라고 생각됩니다.
## Optimizer 를 통한 성능 향상을 확인하기 위해서는 epoch 를 크게 늘려서 과적합 가능성이 있는 상황을 구성해야겠습니다.

Accuracy of the network on the 10000 test images :55 %


In [6]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

### (7) 화면 출력 확인 및 부류별 분류기의 성능 서술

In [7]:
## 부류별 분류기의 성능도 여전히 특정 개체에 대해서만 높은 성능을 가지며,
## 앞에서와 큰 차이가 없다고 볼 수 있습니다.

for i in range(10):
    print('Accuracy of %5s: %2d %%'%(classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane: 55 %
Accuracy of   car: 68 %
Accuracy of  bird: 38 %
Accuracy of   cat: 48 %
Accuracy of  deer: 44 %
Accuracy of   dog: 29 %
Accuracy of  frog: 58 %
Accuracy of horse: 78 %
Accuracy of  ship: 75 %
Accuracy of truck: 61 %
